In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
X = pd.read_csv('train_data.csv')
y = pd.read_csv('train_label.csv')

pred_df = pd.read_csv('predict_proba.csv')

In [3]:
# this is the probability prediction of the X_test set, generated from the "Modeling & Prediction.ipynb" notebook

pred_df.head()

,randomforest_pred,gboost_pred,adaboost_pred,lightgbm_pred,true_label
0,0.300000,0.192014,0.493285,0.237271,0
1,0.286667,0.387071,0.496220,0.394508,0
2,0.270000,0.340027,0.498010,0.261894,0
3,0.310000,0.341312,0.496605,0.291639,0
4,0.000000,0.182456,0.496262,0.028853,0


In [4]:
# the same random split as the modeling 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=695)

X_test.reset_index(inplace=True)
y_test.reset_index(inplace=True)

test_set = X_test.merge(y_test,left_on='index',right_on='index')
test_set.head()

# contains the original index, all the 116 features, and the true label

,index,alpha_count,digit_count,punc_count,stopword_count,verb_count,entity_type,raw_length,token_count,token_count_raw,...,nounchunk_syllables,nonstop_sylla_ratio,pron_sylla_ratio,nonstopN_sylla_ratio,nonstopadj_sylla_ratio,entity_sylla_ratio,nounchunk_sylla_ratio,nonbasic_syllables,nonbasic_sylla_ratio,label
0,291668,14,0,2,8,0,0,68,14,16,...,16,0.571429,0.000000,0.571429,0.000000,0.000000,0.761905,2,0.095238,0
1,406090,12,1,2,7,1,3,67,13,15,...,11,0.555556,0.111111,0.277778,0.000000,0.277778,0.611111,11,0.611111,0
2,384222,25,2,2,14,2,1,167,27,29,...,27,0.666667,0.285714,0.071429,0.142857,0.285714,0.642857,25,0.595238,0
3,242393,15,0,3,7,3,2,92,15,18,...,8,0.625000,0.291667,0.041667,0.000000,0.250000,0.333333,14,0.583333,0
4,367458,21,0,6,9,1,2,101,21,27,...,17,0.720000,0.280000,0.280000,0.200000,0.280000,0.680000,16,0.640000,0


In [5]:
right = []
false = []

for i in range(len(pred_df)):
    
    pred1 = round(pred_df['randomforest_pred'][i],0)
    pred2 = round(pred_df['gboost_pred'][i],0)
    pred3 = round(pred_df['adaboost_pred'][i],0)
    pred4 = round(pred_df['lightgbm_pred'][i],0)
    overall_score = sum([pred1,pred2,pred3,pred4])
    
    actual = pred_df['true_label'][i]
    
    if overall_score == 4:
        if actual == 1:
            right.append(i)
        else:
            false.append(i)
        
    elif overall_score == 0:
        if actual == 0:
            right.append(i)
        else:
            false.append(i) 
            
false[:5]

# the X_test-based indexes for the falsely classified instances 
# this is different from the original index

[8, 16, 21, 28, 32]

In [6]:
test_set.rename({'index': 'original_index'}, axis='columns',inplace=True)
test_set.reset_index(inplace = True)
test_set.head()

,index,original_index,alpha_count,digit_count,punc_count,stopword_count,verb_count,entity_type,raw_length,token_count,...,nounchunk_syllables,nonstop_sylla_ratio,pron_sylla_ratio,nonstopN_sylla_ratio,nonstopadj_sylla_ratio,entity_sylla_ratio,nounchunk_sylla_ratio,nonbasic_syllables,nonbasic_sylla_ratio,label
0,0,291668,14,0,2,8,0,0,68,14,...,16,0.571429,0.000000,0.571429,0.000000,0.000000,0.761905,2,0.095238,0
1,1,406090,12,1,2,7,1,3,67,13,...,11,0.555556,0.111111,0.277778,0.000000,0.277778,0.611111,11,0.611111,0
2,2,384222,25,2,2,14,2,1,167,27,...,27,0.666667,0.285714,0.071429,0.142857,0.285714,0.642857,25,0.595238,0
3,3,242393,15,0,3,7,3,2,92,15,...,8,0.625000,0.291667,0.041667,0.000000,0.250000,0.333333,14,0.583333,0
4,4,367458,21,0,6,9,1,2,101,21,...,17,0.720000,0.280000,0.280000,0.200000,0.280000,0.680000,16,0.640000,0


In [7]:
correct = test_set[test_set['index'].isin(right)]
wrong = test_set[test_set['index'].isin(false)]

correct.shape, wrong.shape

((47542, 119), (14538, 119))

In [8]:
(47542 + 14538) / X_test.shape[0]

0.7447752957266598

#### It actually shocked me a bit : for almost 75% of the instances the 4 algorithms make the exact same prediction.

In [9]:
wrong.head()

,index,original_index,alpha_count,digit_count,punc_count,stopword_count,verb_count,entity_type,raw_length,token_count,...,nounchunk_syllables,nonstop_sylla_ratio,pron_sylla_ratio,nonstopN_sylla_ratio,nonstopadj_sylla_ratio,entity_sylla_ratio,nounchunk_sylla_ratio,nonbasic_syllables,nonbasic_sylla_ratio,label
8,8,223945,19,0,1,10,4,0,120,19,...,22,0.756757,0.000000,0.486486,0.081081,0.000000,0.594595,19,0.513514,0
16,16,98863,21,0,2,12,0,0,134,21,...,22,0.775000,0.075000,0.300000,0.400000,0.000000,0.550000,28,0.700000,1
21,21,40680,24,1,4,9,1,2,157,25,...,28,0.767442,0.441860,0.093023,0.046512,0.558140,0.651163,26,0.604651,1
28,28,335091,18,0,3,6,1,5,118,18,...,22,0.638889,0.416667,0.083333,0.055556,0.611111,0.611111,17,0.472222,0
32,32,225491,34,2,4,15,4,6,202,36,...,24,0.773585,0.301887,0.018868,0.094340,0.377358,0.452830,37,0.698113,0


In [10]:
indexes = wrong.sample(n=20)['original_index'].values
indexes

# these are the original indexes of random selected falsely predicted instances (by all algorithms)

array([185096,  26281, 308466, 251207, 309267,  23215, 270882,   4505,
       399221, 152961, 109803,  33556, 206961, 342017, 295571, 351736,
       408418,  62904, 111366, 203728])

In [11]:
original_df = pd.read_csv('WikiLarge_Train.csv')

for i in indexes:
    print(original_df['original_text'][i])
    print('true label : ',original_df['label'][i])
    print(' ')

The nine sailors killed are the last casualties of World War I.
true label :  1
 
Amis was born in Swansea , South Wales .
true label :  1
 
Baron Georges LÃ opold ChrÃ tien FrÃ dÃ ric Dagobert Cuvier -LRB- 23 August 1769 â '' 13 May 1832 -RRB- was a French naturalist and zoologist .
true label :  0
 
Other common names include '' jibÃ ia '' -LRB- Brazil -RRB- and '' macajuel '' -LRB- Trinidadian , pronounced mah-cah-well -RRB- .
true label :  0
 
Nikita Sergeyevich Khrushchyov -LRB- Russian : Ð Ð Ð Ð Ì Ñ Ð Ð Ð Ñ Ð Ð Ì Ð Ð Ð Ñ Ð Ñ Ñ Ñ Ñ ` Ð -RRB- IPA : -LRB- xruË É Ë Ê of -RRB- , commonly spelled in English as Khrushchev -RRB- April 17 , 1894 â '' September 11 , 1971 -RRB- was the leader of the Soviet Union after the death of Joseph Stalin .
true label :  0
 
Often tell-tales , small pieces of yarn , are attached to the sail .
true label :  1
 
Her husband , Pompeianus , was not involved , but two men , alleged to have been her lovers , attempted to murder Commodus as he entered the th

### From above, I personally think a lot of the instances have the wrong label... 

#### or should I say, for some sentences the true label values are quite debatable.

In [12]:
# Let's do another round

indexes1 = wrong.sample(n=20,random_state = 42)['original_index'].values
indexes1

array([175321, 159282, 400244, 268433, 267430, 262314, 217073,  74053,
       281765, 335106, 262284, 334270, 176649,  58639, 127893, 364381,
       297941, 381870, 269198, 121511])

In [13]:
for i in indexes1:
    print(original_df['original_text'][i])
    print('True label : ',original_df['label'][i])
    print(' ')

She was sworn in on January 23 , 1997 .
True label :  1
 
A short dark mane grows along the neck , and males also sport a black mask and black patches on the sides of the neck .
True label :  1
 
John Stuart Mill , in his work , On Liberty , was the first to recognize the difference between liberty as the freedom to act and liberty as the absence of coercion -LRB- being forced to do something -RRB- .
True label :  0
 
Boveri reasoned that a cancerous tumor begins with a single cell in which the make up of its chromosomes becomes scrambled , causing the cells to divide uncontrollably .
True label :  0
 
Other websites Theoi Project , Ares information on Ares from classical literature , images from Greek and Roman art .
True label :  0
 
In international sports , Australia has very strong teams in cricket , hockey , netball , rugby league , rugby union , and performs well in cycling , rowing and swimming .
True label :  0
 
Silt is easily transported in water and is fine enough to be car